In [4]:
import pandas as pd

df = pd.read_csv("../experimental_records.csv")
df[4:]

,date,model_name,test_f1_score,description
4,2024-02-05/17:56:50,decision_tree_w_oversampling,0.391892,모든 feature 사용/전처리 X/RandomOverSampler 적용/결측치는 ...
5,2024-02-05/22:53:27,decision_tree_w_undersampling,0.586722,모든 feature 사용/전처리 X/RandomUnderSampler 적용/결측치는...
6,2024-02-06/00:00:20,ensemble_decisiontree_w_undersampling,0.652855,10개 decision tree 사용/seed33~42/mean validation...
7,2024-02-06/11:45:07,ensemble_decisiontree_w_undersampling,0.649734,20개 decision tree 사용/seed33~52/mean validation...
8,2024-02-06/14:51:02,decision_tree_w_undersampling_delete_feature,0.610570,"""customer_country.1"" feature 삭제/RandomUnderSam..."
9,2024-02-06/15:02:31,decision_tree_w_undersampling_delete_features,0.581432,"결측치 비율이 80% 이상인 feature 및 ""customer_country.1""..."
10,2024-02-06/15:12:31,decision_tree_w_undersampling_feature_normaliz...,0.554104,"""customer_country"" 정규화/""customer_country.1"" fe..."
11,2024-02-06/15:20:02,ensemble_decisiontree_w_undersampling_and_dele...,0.638188,"10개 decision tree 사용/""customer_country.1"" feat..."


### Sampler를 이용한 실험 결과

**A. <span style='color:blue'>[0.391892]</span> Oversampling만 하는 것은 무리**
- Negative sample이 positive sample보다 10배 이상 많은 상황에서 oversampling을 하니, training data에 overfitting이 발생한 것 같고, 그에 따라 `test_f1_score`가 큰 폭으로 하락하였음.
---
**B. <span style='color:red'>[0.586722]</span> Undersampling 효과 확인 + data imbalance 문제 해소**
- 반대로 negative sample을 undersampling하여 positive sample과 비율을 맞춰준 뒤 학습한 모델을 제출하니, `test_f1_score`가 큰 폭으로 상승하였음.
---
<span style='color:tomato'>**C. [0.652855, Best] Ensemble decision tree + undersampling**</span>
- Negative sample을 undersampling 하는 과정에서 버려지는 데이터가 많았기 때문에, 여러 seed를 활용하여 서로 다른 negative sample을 가지는 subset을 만든 뒤, 이를 ensemble 해보자는 아이디어를 기반으로 실험한 결과 `test_f1_score`가 큰 폭으로 상승하였음.
---
**D. <span style='color:red'>[0.610570]</span> `customer_country.1` feature 삭제에 따른 성능 향상 확인**
- Ensemble 모델에 feature preprocessing을 적용하기 이전에, decision tree with undersampling 세팅에 feature preprocessing을 적용하여 성능을 테스트하고자 했고, 총 세 가지 전처리 시나리오를 수행하였음.
    1. `customer_country.1` feature 삭제
    1. `customer_country.1` feature 삭제 + 결측치 비율이 80% 이상인 feature 삭제
    1. `customer_country.1` feature 삭제 + `customer_country` feature 정규화
    - 실험 B와 결과를 비교했을 때, test_f1_score 기준 1번은 +0.03, 나머지는 하락
---
**E. <span style='color:blue'>[0.638188]</span> Feature preprocessing 효과는 없었음**
- 실험 D-1: `customer_country.1` feature 삭제에 따른 성능 향상을 관측했기 때문에, 현재 가장 높은 성능을 기록 중인 실험 C에 실험 D-1 세팅을 추가
- Best test f1 score 대비 0.02 하락 -> feature preprocessing은 나중에 하는 것으로

### 실험 계획

- Undersampling + 다른 모델 (e.g., svm, gbm, AdaBoost, lightGBM, XGBoost, CatBoost)
- 새로운 기법 탐색 (e.g., Logistic Regression, Out-Of-Distribution Detection)